# REINFORCE in pytorch

Just like we did before for q-learning, this time we'll design a pytorch network to learn `CartPole-v0` via policy gradient (REINFORCE).

Most of the code in this notebook is taken from approximate qlearning, so you'll find it more or less familiar and even simpler.

In [45]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/spring20/setup_colab.sh -O- | bash

    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/grading.py -O ../grading.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/week5_policy_based/submit.py

    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [46]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v1", render_mode="rgb_array")

# gym compatibility: unwrap TimeLimit
if hasattr(env, '_max_episode_steps'):
    env = env.env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

# Building the network for REINFORCE

For REINFORCE algorithm, we'll need a model that predicts action probabilities given states. Let's define such a model below.

In [47]:
import torch
import torch.nn as nn

In [48]:
# Build a simple neural network that predicts policy logits. 
# Keep it simple: CartPole isn't worth deep architectures.
model = nn.Sequential(
  nn.Linear(in_features=4, out_features=256, bias=True)
  , nn.ReLU()
  , nn.Linear(in_features=256, out_features=128, bias=True)
  , nn.ReLU()
  , nn.Linear(in_features=128, out_features=n_actions, bias=True)
)
cuda = torch.device('cuda') 
model.cuda()

Sequential(
  (0): Linear(in_features=4, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=2, bias=True)
)

#### Predict function

Note: output value of this function is not a torch tensor, it's a numpy array.
So, here gradient calculation is not needed.
<br>
Use [no_grad](https://pytorch.org/docs/stable/autograd.html#torch.autograd.no_grad)
to suppress gradient calculation.
<br>
Also, `.detach()` (or legacy `.data` property) can be used instead, but there is a difference:
<br>
With `.detach()` computational graph is built but then disconnected from a particular tensor,
so `.detach()` should be used if that graph is needed for backprop via some other (not detached) tensor;
<br>
In contrast, no graph is built by any operation in `no_grad()` context, thus it's preferable here.

In [49]:
from scipy.special import softmax
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

@torch.no_grad()
def predict_probs(states):
    """ 
    Predict action probabilities given states.
    :param states: numpy array of shape [batch, state_shape]
    :returns: numpy array of shape [batch, n_actions]
    """
    # convert states, compute logits, use softmax to get probability
    "<YOUR CODE>"
    n = states.shape[0]
    states = torch.tensor(states, dtype=torch.float32, device=cuda)
    qvalues = model(states) 
    qvalues_proba = nn.functional.softmax(qvalues, dim=1)

    
    return qvalues_proba.cpu().detach().numpy()

In [50]:
print(env.reset())

(array([0.03570739, 0.02011935, 0.03812407, 0.0411163 ], dtype=float32), {})


In [51]:
test_states = np.array([env.reset()[0] for _ in range(5)])
test_probas = predict_probs(test_states)
assert isinstance(
    test_probas, np.ndarray), "you must return np array and not %s" % type(test_probas)
assert tuple(test_probas.shape) == (
    test_states.shape[0], env.action_space.n), "wrong output shape: {}".format(np.shape(test_probas))
assert np.allclose(np.sum(test_probas, axis=1),
                   1), "probabilities do not sum to 1"

### Play the game

We can now use our newly built agent to play the game.

In [52]:
def generate_session(env, t_max=1000):
    """ 
    play a full session with REINFORCE agent and train at the session end.
    returns sequences of states, actions andrewards
    """
    # arrays to record session
    states, actions, rewards = [], [], []
    s = env.reset()[0]

    for t in range(t_max):
        # action probabilities array aka pi(a|s)
        action_probs = predict_probs(np.array([s]))[0]

        # Sample action with given probabilities.
        "<YOUR CODE>"
        a = np.random.choice((0, 1), p=action_probs)
        new_s, r, terminated, truncated, info = env.step(a)

        # record session history to train later
        states.append(s)
        actions.append(a)
        rewards.append(r)

        s = new_s
        if terminated or truncated:
            break

    return states, actions, rewards

In [53]:
# test it
states, actions, rewards = generate_session(env)


### Computing cumulative rewards

In [54]:
def get_cumulative_rewards(rewards,  # rewards at each step
                           gamma=0.99  # discount for reward
                           ):
    """
    take a list of immediate rewards r(s,a) for the whole session 
    compute cumulative returns (a.k.a. G(s,a) in Sutton '16)
    G_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...

    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute G_t = r_t + gamma*G_{t+1} recurrently

    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    # <YOUR CODE>
    l = len(rewards)
    G = list(range(l))

    G[l-1] = rewards[-1]

    for t in reversed(range(l-1)):
      G[t] = rewards[t] + gamma * G[t+1]
   
  
    return G

In [55]:
get_cumulative_rewards(rewards)
assert len(get_cumulative_rewards(list(range(100)))) == 100
assert np.allclose(get_cumulative_rewards([0, 0, 1, 0, 0, 1, 0], gamma=0.9), [
                   1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards(
    [0, 0, 1, -2, 3, -4, 0], gamma=0.5), [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards(
    [0, 0, 1, 2, 3, 4, 0], gamma=0), [0, 0, 1, 2, 3, 4, 0])
print("looks good!")

looks good!


#### Loss function and updates

We now need to define objective and update over policy gradient.

Our objective function is

$$ J \approx  { 1 \over N } \sum_{s_i,a_i} G(s_i,a_i) $$


Following the REINFORCE algorithm, we can define our objective as follows: 

$$ \hat J \approx { 1 \over N } \sum_{s_i,a_i} \log \pi_\theta (a_i \mid s_i) \cdot G(s_i,a_i) $$

When you compute gradient of that function over network weights $ \theta $, it will become exactly the policy gradient.

In [56]:
def to_one_hot(y_tensor, ndims):
    """ helper: take an integer vector and convert it to 1-hot matrix. """
    y_tensor = y_tensor.type(torch.cuda.
                             LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims, device=cuda).scatter_(1, y_tensor, 1)
    return y_one_hot

In [57]:
# Your code: define optimizers
optimizer = torch.optim.Adam(model.parameters(), 1e-3)


def train_on_session(states, actions, rewards, gamma=0.99, entropy_coef=1e-2):
    """
    Takes a sequence of states, actions and rewards produced by generate_session.
    Updates agent's weights by following the policy gradient above.
    Please use Adam optimizer with default parameters.
    Confer: https://fosterelli.co/entropy-loss-for-reinforcement-learning
    """

    # cast everything into torch tensors
    states = torch.tensor(states, dtype=torch.float32, device=cuda)
    actions = torch.tensor(actions, dtype=torch.int32, device=cuda)
    cumulative_returns = np.array(get_cumulative_rewards(rewards, gamma))
    cumulative_returns = torch.tensor(cumulative_returns, dtype=torch.float32, device=cuda)

    # predict logits, probas and log-probas using an agent.
    logits = model(states)
    probs = nn.functional.softmax(logits, -1)
    log_probs = nn.functional.log_softmax(logits, -1)

    assert all(isinstance(v, torch.Tensor) for v in [logits, probs, log_probs]), \
        "please use compute using torch tensors and don't use predict_probs function"

    # select log-probabilities for chosen actions, log pi(a_i|s_i)
    log_probs_for_actions = torch.sum(
        log_probs * to_one_hot(actions, env.action_space.n), dim=1)

   
    # Compute loss here. Don't forgen entropy regularization with `entropy_coef` 
    "<YOUR CODE>"
    entropy = -torch.sum(log_probs * probs).to(cuda)
    #print(entropy)
    "<YOUR CODE>"
    J_hat = torch.mean(log_probs_for_actions * cumulative_returns).to(cuda)
    # maximize J value, minimize entropy
    loss = - J_hat + entropy_coef * entropy   # this implementation does not use baseline


    # Gradient descent step
    "<YOUR CODE>"
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # technical: return session rewards to print them later
    return np.sum(rewards)

### The actual training

In [59]:
for i in range(100):
    rewards = [train_on_session(*generate_session(env))
               for _ in range(100)]  # generate new sessions
    print("mean reward:%.3f" % (np.mean(rewards)))
    if np.mean(rewards) > 300:
        print("You Win!")  # but you can train even further
        break

mean reward:12.590
mean reward:9.400
mean reward:9.320
mean reward:9.320
mean reward:9.160
mean reward:9.260
mean reward:9.320
mean reward:9.360
mean reward:9.410
mean reward:9.320
mean reward:9.390
mean reward:9.310
mean reward:9.380
mean reward:9.260
mean reward:9.420
mean reward:9.300
mean reward:9.380
mean reward:9.330
mean reward:9.330
mean reward:9.400
mean reward:9.320
mean reward:9.340
mean reward:9.270
mean reward:9.350
mean reward:9.490
mean reward:9.470
mean reward:9.240
mean reward:9.400
mean reward:9.300
mean reward:9.270
mean reward:9.490
mean reward:9.260
mean reward:9.370
mean reward:9.430
mean reward:9.370
mean reward:9.420
mean reward:9.380
mean reward:9.290
mean reward:9.290
mean reward:9.410
mean reward:9.410
mean reward:9.280


KeyboardInterrupt: 

### Video

In [ ]:
# Record sessions

import gym.wrappers

with gym.wrappers.Monitor(gym.make("CartPole-v0"), directory="videos", force=True) as env_monitor:
    sessions = [generate_session(env_monitor) for _ in range(100)]

ModuleNotFoundError: No module named 'gym'

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from pathlib import Path
from IPython.display import HTML

video_names = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-1]))  # You can also try other indices

In [ ]:
from submit import submit_cartpole_pytorch
submit_cartpole_pytorch(generate_session, 'agimenezzapiola@gmail.com', 'tZveBlKsoIrQin6Z')

Submitted to Coursera platform. See results on assignment page!


That's all, thank you for your attention!

Not having enough? There's an actor-critic waiting for you in the honor section. But make sure you've seen the videos first.